In [4]:
pip install -U langchain-community

  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached pydantic_settings-2.7.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   -------------------- ------------------- 1.3/2.5 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 8.0 MB/s eta 0:00:00
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached pydantic_settings-2.7.1-py3-none-any.whl (29 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from tqdm.autonotebook import tqdm


c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
PINECONE_API_KEY = "pcsk_2o8g2_F1QiRhz5R9HdvmjmfnrYByTPtY1BUFFED3JAtcs13SgjYdqfJYhcNbbAexhgCQx"
PINECONE_API_ENV = "gcp-starter"

In [7]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [8]:
extracted_data = load_pdf("data/")

In [10]:
# extracted_data

In [9]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5860


In [11]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [12]:
from transformers import AutoModel, AutoTokenizer
import torch

# Load pre-trained model and tokenizer from Hugging Face
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example text
sentences = ["Hello World"]
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

# Get embeddings
with torch.no_grad():
    embeddings = model(**inputs).last_hidden_state.mean(dim=1)

print(embeddings)


tensor([[-1.9745e-01,  1.7767e-01,  3.8570e-02,  1.4952e-01, -2.2542e-01,
         -9.1803e-01,  3.8326e-01, -3.6890e-02, -2.7174e-01,  8.4522e-02,
          4.0589e-01,  3.1800e-01,  1.0992e-01, -1.5034e-01, -5.7896e-02,
         -1.5428e-01,  1.2775e-01, -1.2729e-01, -8.5727e-01, -1.0018e-01,
          4.3961e-02,  3.1127e-01,  1.8638e-02,  1.8169e-01, -4.8461e-01,
         -1.6840e-01,  2.9548e-01,  2.7560e-01, -1.8983e-02, -3.3376e-01,
          2.4035e-01,  1.2720e-01,  7.3412e-01, -1.2793e-01, -6.6753e-02,
          3.6038e-01, -1.8828e-01, -5.2244e-01, -1.7854e-01,  3.0180e-01,
          2.6936e-01, -4.8222e-01, -1.7213e-01, -1.1880e-01,  5.4508e-02,
         -2.1314e-02,  4.2054e-02,  2.2520e-01,  5.3417e-01, -2.1697e-02,
         -3.0205e-01, -3.3249e-01, -3.9311e-02,  3.0257e-02,  4.7472e-01,
          1.1089e-01,  3.5991e-02, -5.9163e-02,  5.1727e-02, -2.1581e-01,
         -2.5889e-01,  1.3754e-01, -3.9768e-02,  7.7265e-02,  5.7300e-01,
         -4.1052e-01, -1.2424e-01,  1.

In [13]:
from transformers import AutoModel, AutoTokenizer
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load pre-trained model and tokenizer from Hugging Face
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example text (replace with your text chunks)
text_chunks = [
    {"page_content": "Allergies are the body's immune response to substances."},
    {"page_content": "A headache can occur from various conditions, including allergies."},
    {"page_content": "Fever is a common symptom of infection."},
    {"page_content": "Skin rashes may appear when someone has an allergy."},
    {"page_content": "Treatment for allergies often involves antihistamines."}
]

# Step 1: Generate embeddings for the text chunks
def generate_embeddings(text_chunks):
    embeddings = []
    for chunk in text_chunks:
        sentence = chunk['page_content']
        inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")

        # Get embeddings
        with torch.no_grad():
            embedding = model(**inputs).last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings.append(embedding)
    return np.array(embeddings)

# Get embeddings for the text chunks
embeddings = generate_embeddings(text_chunks)

# Step 2: Perform similarity search
def similarity_search(query, embeddings, text_chunks, top_k=3):
    # Generate embedding for the query
    inputs = tokenizer(query, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        query_embedding = model(**inputs).last_hidden_state.mean(dim=1).squeeze().numpy()

    # Compute cosine similarities between query and document embeddings
    similarities = cosine_similarity([query_embedding], embeddings)
    
    # Get indices of the top_k most similar documents
    top_k_indices = similarities.argsort()[0][-top_k:][::-1]
    
    # Get the corresponding documents
    results = [text_chunks[i] for i in top_k_indices]
    
    return results

# Step 3: Query
query = "What are allergies?"
results = similarity_search(query, embeddings, text_chunks)

# Output the results
print("Results:")
for result in results:
    print(result['page_content'])


Results:
Allergies are the body's immune response to substances.
Treatment for allergies often involves antihistamines.
Skin rashes may appear when someone has an allergy.


In [ ]:
from transformers import AutoModel, AutoTokenizer, GPT2LMHeadModel, GPT2Tokenizer
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load pre-trained model and tokenizer from Hugging Face
model_name = "sentence-transformers/all-MiniLM-L6-v2"

# Use CPU to avoid memory issues
try:
    model = AutoModel.from_pretrained(model_name).to('cpu')
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print(f"Model and tokenizer loaded successfully on CPU.")
except Exception as e:
    print(f"Error loading model or tokenizer: {e}")
    raise

# Example text (replace with your text chunks)
text_chunks = [
    {"page_content": "Allergies are the body's immune response to substances."},
    {"page_content": "A headache can occur from various conditions, including allergies."},
    {"page_content": "Fever is a common symptom of infection."},
    {"page_content": "Skin rashes may appear when someone has an allergy."},
    {"page_content": "Treatment for allergies often involves antihistamines."}
]

# Generate embeddings for the text chunks
def generate_embeddings(text_chunks):
    embeddings = []
    for chunk in text_chunks:
        sentence = chunk['page_content']
        inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")

        # Get embeddings
        with torch.no_grad():
            embedding = model(**inputs).last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings.append(embedding)
    return np.array(embeddings)

# Get embeddings for the text chunks
embeddings = generate_embeddings(text_chunks)

# Perform similarity search
def similarity_search(query, embeddings, text_chunks, top_k=3):
    # Generate embedding for the query
    inputs = tokenizer(query, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        query_embedding = model(**inputs).last_hidden_state.mean(dim=1).squeeze().numpy()

    # Compute cosine similarities between query and document embeddings
    similarities = cosine_similarity([query_embedding], embeddings)
    
    # Get indices of the top_k most similar documents
    top_k_indices = similarities.argsort()[0][-top_k:][::-1]
    
    # Get the corresponding documents
    results = [text_chunks[i] for i in top_k_indices]
    
    return results

# Function to generate an answer from context using a model (adjusted for quality)
def generate_answer(context, question):
    # Construct a better prompt that encourages more meaningful answers
    prompt = f"""
    Answer the following question based on the context provided below. Provide a detailed, informative response. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context: {context}
    Question: {question}
    
    Only return the helpful answer below and nothing else.
    Helpful answer:
    """
    
    # Example: Use GPT-2 for answering (can replace with another model)
    gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to('cpu')
    gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

    # Adjust temperature and use top-p sampling to control repetition and make the output more varied
    inputs = gpt2_tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    
    # Generate with adjusted temperature and top-p
    outputs = gpt2_model.generate(
        inputs["input_ids"],
        max_length=200,  # Allow for longer responses
        temperature=0.7,  # Less repetitive, more creative
        top_p=0.9,  # Top-p sampling to prevent overly deterministic output
        num_return_sequences=1,
        no_repeat_ngram_size=2,  # Prevent repetition of n-grams
        early_stopping=True
    )
    
    # Decode and clean the output
    answer = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return answer

# Main query function to get the helpful answer
def get_helpful_answer(query):
    # Retrieve relevant documents (context) based on the query
    results = similarity_search(query, embeddings, text_chunks)
    
    # Format the context for the prompt (combine the text from the top results)
    context = "\n".join([result['page_content'] for result in results])
    
    # Generate the answer from the context
    try:
        response = generate_answer(context, query)
        return response
    except Exception as e:
        print(f"Error generating response: {e}")
        return "Sorry, there was an error processing your request."

# Main loop to query and get answers
while True:
    user_input = input("Input Prompt: ")
    result = get_helpful_answer(user_input)
    print("Response: ", result)


Model and tokenizer loaded successfully on CPU.


c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\transformers\generation\configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask a

Response:  
    Answer the following question based on the context provided below. Provide a detailed, informative response. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context: Treatment for allergies often involves antihistamines.
Allergies are the body's immune response to substances.
Skin rashes may appear when someone has an allergy.
    Question: Medicine for allergy
    
    Only return the helpful answer below and nothing else.
    Helpful answer:
     I have a history of allergies. I am allergic to peanuts, peanuts and other allergens. My doctor has told me that I should not use peanuts. If I do, I will not be able to use them. The doctor also told my family that they should be careful about using peanuts in their food. They have told them that if


c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\transformers\generation\configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask a

Response:  
    Answer the following question based on the context provided below. Provide a detailed, informative response. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context: Skin rashes may appear when someone has an allergy.
Allergies are the body's immune response to substances.
Treatment for allergies often involves antihistamines.
    Question: what are allergy?
    
    Only return the helpful answer below and nothing else.
    Helpful answer:
     The following is a list of the most common allergies. The list is not exhaustive. If there are any allergies, please let me know. I will try my best to help you. Please note that I am not a doctor. My experience is that most people with allergies are not allergic to anything. They are allergic only to the


c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\transformers\generation\configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask a

Response:  
    Answer the following question based on the context provided below. Provide a detailed, informative response. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context: Skin rashes may appear when someone has an allergy.
Allergies are the body's immune response to substances.
Treatment for allergies often involves antihistamines.
    Question: What are allergy?
    
    Only return the helpful answer below and nothing else.
    Helpful answer:
     The following is a list of the most common allergies. The list is not exhaustive. If there are any allergies, please let me know. I will try my best to help you. Please note that I am not a doctor. My experience is that most people with allergies are not allergic to anything. They are allergic only to the


c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\transformers\generation\configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask a

Response:  
    Answer the following question based on the context provided below. Provide a detailed, informative response. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context: Fever is a common symptom of infection.
Allergies are the body's immune response to substances.
Treatment for allergies often involves antihistamines.
    Question: escape
    
    Only return the helpful answer below and nothing else.
    Helpful answer:
     You can return to the original question. If the question is not answered, you can ask the person who answered it. The person will respond with a brief explanation of the situation. You may also ask for a copy of your medical history. This information will help you determine if you have a medical condition. It is important to remember that the information you provide is


In [ ]:
from transformers import AutoModel, AutoTokenizer, GPT2LMHeadModel, GPT2Tokenizer
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load pre-trained model and tokenizer from Hugging Face
model_name = "sentence-transformers/all-MiniLM-L6-v2"

# Use CPU to avoid memory issues
try:
    model = AutoModel.from_pretrained(model_name).to('cpu')
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print(f"Model and tokenizer loaded successfully on CPU.")
except Exception as e:
    print(f"Error loading model or tokenizer: {e}")
    raise

# Example text (replace with your text chunks)
text_chunks = [
    {"page_content": "Allergies are the body's immune response to substances."},
    {"page_content": "A headache can occur from various conditions, including allergies."},
    {"page_content": "Fever is a common symptom of infection."},
    {"page_content": "Skin rashes may appear when someone has an allergy."},
    {"page_content": "Treatment for allergies often involves antihistamines."}
]

# Generate embeddings for the text chunks
def generate_embeddings(text_chunks):
    embeddings = []
    for chunk in text_chunks:
        sentence = chunk['page_content']
        inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")

        # Get embeddings
        with torch.no_grad():
            embedding = model(**inputs).last_hidden_state.mean(dim=1).squeeze().numpy()
        embeddings.append(embedding)
    return np.array(embeddings)

# Get embeddings for the text chunks
embeddings = generate_embeddings(text_chunks)

# Perform similarity search
def similarity_search(query, embeddings, text_chunks, top_k=3):
    # Generate embedding for the query
    inputs = tokenizer(query, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        query_embedding = model(**inputs).last_hidden_state.mean(dim=1).squeeze().numpy()

    # Compute cosine similarities between query and document embeddings
    similarities = cosine_similarity([query_embedding], embeddings)
    
    # Get indices of the top_k most similar documents
    top_k_indices = similarities.argsort()[0][-top_k:][::-1]
    
    # Get the corresponding documents
    results = [text_chunks[i] for i in top_k_indices]
    
    return results

# Function to generate an answer from context using a model (adjusted for quality)
def generate_answer(context, question):
    # Construct a better prompt that encourages meaningful answers and clarifies the response
    prompt = f"""
    Answer the following question based on the context provided below. Provide a clear, informative, and concise response.
    Avoid unnecessary details and repetition. If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context: {context}
    Question: {question}
    
    Only return the helpful answer below and nothing else.
    Helpful answer:
    """
    
    # Use GPT-2 for answering (can replace with another model)
    gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to('cpu')
    gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

    # Adjust temperature and use top-p sampling to control repetition and make the output more varied
    inputs = gpt2_tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
    
    # Generate with adjusted temperature and top-p
    outputs = gpt2_model.generate(
        inputs["input_ids"],
        max_length=200,  # Allow for longer responses
        temperature=0.7,  # Less repetitive, more creative
        top_p=0.85,  # Top-p sampling to prevent overly deterministic output
        num_return_sequences=1,
        no_repeat_ngram_size=2,  # Prevent repetition of n-grams
        early_stopping=True
    )
    
    # Decode and clean the output
    answer = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Check if the answer is meaningful; if it’s too vague, fallback to a default message
    if "I don't know" in answer or len(answer.split()) < 3:
        return "Sorry, I don't know the answer."
    
    return answer

# Main query function to get the helpful answer
def get_helpful_answer(query):
    # Retrieve relevant documents (context) based on the query
    results = similarity_search(query, embeddings, text_chunks)
    
    # Format the context for the prompt (combine the text from the top results)
    context = "\n".join([result['page_content'] for result in results])
    
    # Generate the answer from the context
    try:
        response = generate_answer(context, query)
        return response
    except Exception as e:
        print(f"Error generating response: {e}")
        return "Sorry, there was an error processing your request."

# Main loop to query and get answers
while True:
    user_input = input("Input Prompt: ")
    result = get_helpful_answer(user_input)
    print("Response: ", result)


c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model and tokenizer loaded successfully on CPU.


c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.85` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\ASUS\anaconda3\install\envs\mchatbot\Lib\site-packages\transformers\generation\configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The attention mask 

Response:  
    Answer the following question based on the context provided below. Provide a clear, informative, and concise response.
    Avoid unnecessary details and repetition. If you don't know the answer, just say that you don't know, don't try to make up an answer.

    Context: Skin rashes may appear when someone has an allergy.
Allergies are the body's immune response to substances.
A headache can occur from various conditions, including allergies.
    Question: what is acne?
    
    Only return the helpful answer below and nothing else.
    Helpful answer:
     Skin rash is a common skin condition. It is caused by a combination of factors, such as:  a combination or combination acne,    erythema, or ichthyosis.  The skin rash can be caused from: a) a lack of sun protection
